# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup 
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,0.76,76,100,2.13,GS,1704177046
1,1,port-aux-francais,-49.3500,70.2167,4.80,76,56,16.77,TF,1704177046
2,2,lianyuan,27.6883,111.6642,9.02,84,100,1.94,CN,1704177048
3,3,whitehorse,60.7161,-135.0538,-13.57,92,100,2.06,CA,1704177048
4,4,hawaiian paradise park,19.5933,-154.9731,22.15,90,75,2.57,US,1704177049


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.9,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df.loc[(city_data_df['Max Temp'] <27) & (city_data_df['Max Temp'] >21)&\
                    (city_data_df['Wind Speed'] <4.5)& (city_data_df['Cloudiness']==0) ]

# Drop any rows with null values
new_ideal_city_data_df = ideal_city_data_df.dropna(axis=0 , how="any")

# Display sample data
new_ideal_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
82,82,conde,-7.2597,-34.9075,22.70,100,0,2.06,BR,1704177108
169,169,champerico,14.3000,-91.9167,24.79,78,0,1.22,GT,1704177172
241,241,tura,25.5198,90.2201,24.26,36,0,1.58,IN,1704177221
263,263,pisco,-13.7000,-76.2167,22.03,88,0,3.60,PE,1704177238
322,322,acapulco de juarez,16.8634,-99.8901,22.90,78,0,1.54,MX,1704177140


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
selected_columns = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
hotel_df = new_ideal_city_data_df[selected_columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
82,conde,BR,-7.2597,-34.9075,100,
169,champerico,GT,14.3000,-91.9167,78,
241,tura,IN,25.5198,90.2201,36,
263,pisco,PE,-13.7000,-76.2167,88,
322,acapulco de juarez,MX,16.8634,-99.8901,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 1

params = {"categories":'accommodation.hotel',
          "limit":limit,
          "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"] 
    longitude = hotel_df.loc[index, "Lng"] 

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
         #If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
conde - nearest hotel: Paladino
champerico - nearest hotel: Hotel y Restaurante El Submarino
tura - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
acapulco de juarez - nearest hotel: Hotel del Valle
taunggyi - nearest hotel: Golden Island Cottages
jaisalmer - nearest hotel: Jeet Mahal
brisas de zicatela - nearest hotel: Casa de Olas
disa - nearest hotel: No hotel found
dungarpur - nearest hotel: No hotel found
leteri - nearest hotel: No hotel found
kitale - nearest hotel: Alakara Hotel
sur - nearest hotel: Sur Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
82,conde,BR,-7.2597,-34.9075,100,Paladino
169,champerico,GT,14.3000,-91.9167,78,Hotel y Restaurante El Submarino
241,tura,IN,25.5198,90.2201,36,No hotel found
263,pisco,PE,-13.7000,-76.2167,88,La Portada
322,acapulco de juarez,MX,16.8634,-99.8901,78,Hotel del Valle
386,taunggyi,MM,20.7833,97.0333,28,Golden Island Cottages
403,jaisalmer,IN,26.9147,70.9181,13,Jeet Mahal
438,brisas de zicatela,MX,15.8369,-97.0419,65,Casa de Olas
504,disa,IN,24.2500,72.1667,34,No hotel found
529,dungarpur,IN,23.8333,73.7167,39,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 2,
    color = "City",
    alpha=0.5,
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
map_plot_2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)